# SSZ Colab AutoRunner (Minimal)
This notebook only installs basic dependencies and runs **`run_all_ssz_terminal.py`** exactly **once**.
No other scripts are launched here.

## 0) (Optional) Mount Google Drive
If your project lives in Drive, mount and set `PROJECT_DIR` accordingly. Otherwise skip to **1B** (ZIP upload).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
PROJECT_DIR = '/content/drive/MyDrive/Segmented-Spacetime-Mass-Projection-Unified-Results'  # <--- change if needed
print('PROJECT_DIR =', PROJECT_DIR)

## 1A) Install minimal dependencies
Add or remove packages as needed for your environment.

In [ ]:
%%bash
python -V
pip -q install --upgrade pip
# Minimal set; extend if your runner needs more:
pip -q install numpy pandas scipy matplotlib sympy numba
echo "Deps installed."

## 1B) (Alternative) Upload ZIP of your repo (if not using Drive)
This will unzip to `/content/proj` and set `PROJECT_DIR` accordingly.

In [ ]:
import os, zipfile
from google.colab import files

target = '/content/proj'
os.makedirs(target, exist_ok=True)

print('If you want to upload a ZIP of your project, select it now. Otherwise cancel.')
uploaded = files.upload()
if uploaded:
    zipname = next(iter(uploaded.keys()))
    with zipfile.ZipFile(zipname, 'r') as zf:
        zf.extractall(target)
    print('Unzipped to', target)
    # Try to auto-detect a top-level directory containing the runner
    cand = None
    for root, dirs, files_list in os.walk(target):
        if 'run_all_ssz_terminal.py' in files_list:
            cand = root
            break
    if cand:
        PROJECT_DIR = cand
    else:
        PROJECT_DIR = target
    print('PROJECT_DIR =', PROJECT_DIR)
else:
    print('No upload — using PROJECT_DIR as set above (Drive path or default).')

## 2) Run `run_all_ssz_terminal.py` (single call)
This enforces UTF‑8 I/O and runs the pipeline exactly once.

In [ ]:
import os, sys, subprocess, shlex, pathlib

# Make sure PROJECT_DIR is defined
try:
    PROJECT_DIR
except NameError:
    PROJECT_DIR = '/content'  # fallback

runner = pathlib.Path(PROJECT_DIR) / 'run_all_ssz_terminal.py'
assert runner.exists(), f"Runner not found: {runner}"

env = os.environ.copy()
env["PYTHONUTF8"] = "1"
env["PYTHONIOENCODING"] = "utf-8"
env["PYTHONWARNINGS"] = "ignore"

print("=== EXEC ===")
print("Project dir:", PROJECT_DIR)
print("Runner     :", str(runner))

# Change to project dir and execute once
proc = subprocess.run(
    [sys.executable, str(runner)],
    cwd=PROJECT_DIR,
    env=env,
    text=True,
    capture_output=True,
)

# Stream outputs safely
for label, data in (("STDOUT", proc.stdout), ("STDERR", proc.stderr)):
    if data:
        print(f"--- {label} ---")
        try:
            print(data)
        except UnicodeEncodeError:
            print(data.encode("utf-8", "ignore").decode("utf-8"))

print("Exit code:", proc.returncode)
if proc.returncode != 0:
    raise SystemExit(proc.returncode)

### Notes
- This notebook **only** runs `run_all_ssz_terminal.py`. No other scripts are executed here.
- If your runner needs more packages, add them to the pip cell.
- All UTF‑8 settings are enforced to avoid console encoding issues.